# Práctica Futbol

## Alumno: Óscar Camean Eddine

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen o en su defecto por email.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

-------

# Primera tarea: 
## ¿ En que se diferencian las acciones de las transformaciones en SPARK? ¿Podrías ponerme un par de ejemplo de cada una de ellas y describir que hacen, para que sirve?

Las transformaciones definen el cambio de flujo de la información generando un nuevo RDD, son instrucciones a implantar en los datos.

Cuentan con un corportamiento lazy y se ejecutan cuando se realiza una acción. las tranformaciones principales son:
* map
* Filter
* FilterFlat

Acción nos permite evaluar el RDD y devolver un resultado, algunas acciones son:

* count
* collect
* max
* min
...


-------

# Segunda tarea: 
## Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
data = './download.txt'
raw_data = sc.textFile(data)

In [3]:
raw_data.take(4)

['1::1970-71::1::Athletic Club::Barcelona::1::1::12/09/1970::21942000.0',
 '2::1970-71::1::Las Palmas::Atletico de Madrid::1::1::12/09/1970::21942000.0',
 '3::1970-71::1::Real Madrid::Valencia::2::0::12/09/1970::21942000.0',
 '4::1970-71::1::Celta de Vigo::Sporting de Gijon::2::0::13/09/1970::22028400.0']

In [4]:
data_tratados = raw_data.map(lambda line: line.split('::'))
data_tratados.take(1)

[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

-------

# Tercera tarea: 

## Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

In [5]:
local = data_tratados.filter(lambda line: line[3] == 'Sporting de Gijon') 
#Partidos locales del Sporting

visitante = data_tratados.filter(lambda line: line[4] == 'Sporting de Gijon') 
#Partidos visitantes del Sporting

gol_local = local.map(lambda line: int(line[5])).sum() 
gol_visitante = visitante.map(lambda line: int(line[6])).sum()


goles_totales = gol_local + gol_visitante
print('\nEl número total de goles marcados por el Real Sporting de Gijón es: \n\n',goles_totales)


El número total de goles marcados por el Real Sporting de Gijón es: 

 2042


-------

# Cuarta pregunta: 
## ¿ En que temporada se marcaron más goles?

In [6]:
data_goles = (data_tratados.map(lambda line: (line[1], int(line[5]) + int(line[6])))
              .reduceByKey(lambda a, b: a + b)
              .takeOrdered(1,lambda x: -x[1]))
data_goles

[('2012-13', 2294)]

-----

# Quinta pregunta: 
## ¿Cúal es el equipo que tiene el record de más goles como local?

In [7]:
# Se aplica un map sobre las líneas 3 y la 5 (convertida a entero), 
# se aplica un reduced by key que suma todos los goles. 

goles_max_local = (data_tratados.map(lambda line: (line[3], int(line[5]))) 
            .reduceByKey(lambda a, b: a + b) 
            .takeOrdered(10, lambda x: -x[1])) 

goles_max_local # El record lo tienen el Real Madrid con 2054 goles

[('Real Madrid', 2054),
 ('Barcelona', 2043),
 ('Atletico de Madrid', 1566),
 ('Valencia', 1522),
 ('Athletic Club', 1424),
 ('Real Zaragoza', 1395),
 ('Sevilla', 1384),
 ('Real Sociedad', 1364),
 ('Deportivo', 1343),
 ('Betis', 1328)]

## ¿ y cómo visitante?

In [8]:
goles_max_visitante = (data_tratados.map(lambda line: (line[4], int(line[6])))
            .reduceByKey(lambda a, b: a + b) 
            .takeOrdered(10, lambda x: -x[1]))

goles_max_visitante # como visitante también tiene el record el Real Madrid con 1296

[('Barcelona', 1296),
 ('Real Madrid', 1296),
 ('Atletico de Madrid', 988),
 ('Valencia', 919),
 ('Betis', 853),
 ('Sevilla', 839),
 ('Real Sociedad', 818),
 ('Real Zaragoza', 807),
 ('Athletic Club', 802),
 ('Celta de Vigo', 775)]

------


# Sexta pregunta: 
## ¿Cúales son las 3 décadas en las que más goles se metieron?

In [9]:
# Aplico un map sobre el penultimo string de la linea 7 (década),
# y como valor defino la suma de la linea 5 y 6 (convertidas a enteros), 
# que señala los goles locales y visitantes para, finalmente, ordenar de mayor a menor

decadas_goles = (data_tratados.map(lambda line: (line[7][-2], int(line[5]) + int(line[6])))
                 .reduceByKey(lambda a, b: a + b).
                 takeOrdered(3,lambda x: -x[1]))

decadas_goles #La década de los 2000, los 90 y los 80.

[('0', 20526), ('9', 19321), ('8', 17336)]

----------

#  Séptima pregunta:
## ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

-----------

#  Octava pregunta: 
## ¿Cúal es la media de goles como visitante por partido?

In [10]:
# Se aplica un map sobre la línea 6 convertida a entero y se aplica .mean()

media_goles_visitante = (data_tratados.map(lambda line: int(line[6])).mean())
media_goles_visitante #0.928.

0.9280085752361036

-------------